In [2]:
!pip install -q accelerate peft bitsandbytes transformers trl

In [3]:
import json
import pandas as pd
import textwrap
import os
import torch
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

In [13]:
#add path dataset which you want to test
huggingface_dataset_name = "/kaggle/input/whiiwhii"

dataset = load_dataset(huggingface_dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 559
    })
})

In [5]:
train_test_split = dataset['train'].train_test_split(test_size=0.005)
dataset['test'] = train_test_split['test']
print(f"Test set size: {len(dataset['test'])}") 

Test set size: 3


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id =  "NousResearch/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

# Inference for Shod response

**Load uploded Model from hugging face**

In [8]:
peft_model_dir = "NitinGCJ/ai-quest-p1"  # Replace with actual model name or path

# Load base LLM model and tokenizer
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    # use_auth_token=True  # Add if you need authentication
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_dir)

adapter_config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

In [12]:
# No need to move the model to the device if it's using bitsandbytes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vect=[]
for i in range (0,len(dataset['test'])):
    
    dialogue = dataset['test'][i]['input']
                               
    # Prepare the prompt
    prompt = f"""Act as an expert Analyst with 20+ years of experience\
    in Pharma and Healthcare industry. 

    I have a block of text containing various details related to a patient's background and lifestyle. Please perform Named Entity Recognition (NER) on this text to identify the following Social Determinants of Health (SDoH) entities: 'Education,' 'Occupation,' 'Living Situation,' 'Income Level,' 'Access to Healthcare,' 'Relationship Status,' 'Social Habits' (including smoking and drinking), 'Race,' 'Gender,' and 'Age.' Return each entity in a structured format, such as JSON or a table, where each entity type is clearly labeled.

    OUTPUT_FORMAT: JSON STRING

    EXAMPLE-
    {{
    "Social Determinants of Health":{{
    "Education": "Not Available",
    "Occupation": "Not Available",
    "Living Situation": "Home",
    "Income Level": "Not Available",
    "Access to Healthcare": "Yes",
    "Relationship Status": "Single",
    "Social Habits": {{
    "Smoking": "No",
    "Drinking": "No"
    }},
    "Race": "Black",
    "Gender": "Female",
    "Age": "17"
    }}
    }}

    ### Perform NER from the following:
    {dialogue.strip()}

    ### Response:
    """
    input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()
    outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=200, )
    output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
    print(output)

    vect.append(output)                           


    EDUCATION :
High School Diploma

    OCCUPATION :
Homemaker

    LIVING SITUATION :
Home

    INCOME LEVEL :
Not Available

    ACCESS TO HEALTHCARE :
Yes

    RELATIONSHIP STATUS :
Single

    SOCIAL HABITS :
Smoking : No
Drinking : No

    RACE :
Black

    GENDER :
Female

    AGE :
17

    OTHER :
No

    HISTORY :
Carotid Stenosis, Aortic Stenosis, Hypertension, Hypercholesterolemia, Glaucoma, Uterine Fibroids, Bilateral Carotid Disease, Diabetes, Family History


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


2013-8-27
































































































































































































    {
    "Social Determinants of Health":{
    "Education": "High School",
    "Occupation": "Self Employed",
    "Living Situation": "Home",
    "Income Level": "Above Average",
    "Access to Healthcare": "Yes",
    "Relationship Status": "Married",
    "Social Habits": {
    "Smoking": "No",
    "Drinking": "Occasional"
    },
    "Race": "White",
    "Gender": "Male",
    "Age": "56"
    }
    }

    # Education
    High School

    # Occupation
    Self Employed

    # Living Situation
    Home

    # Income Level
    Above Average

   


In [57]:
final_df = pd.DataFrame(columns=['input', 'output', 'ground_truth'])
for i in range (0,len(vect)):
    final_df.loc[i] = [dataset['test'][i]['input'],json.dumps(vect[i]),dataset['test'][i]['output']]

# output

In [65]:
print("******MODEL-GENRATED******")
print(final_df['ground_truth'][0])
print(final_df['output'][0])
dash_line = '-'.join('' for x in range(100))
print(dash_line)
print("******GROUND_TRUTH******")
print(final_df['ground_truth'][0])


******MODEL-GENRATED******
'json{"age": "80", "gender": "F", "living_situation": "lives alone though her family is in the area", "relationship_status": ""}'
{"Education": "Not Available", "Occupation": "Not Available", "Living Situation": "Home", "Income Level": "Not Available", "Access to Healthcare": "Yes", "Relationship Status": "Single", "Social Habits": "{\"Smoking\":\"No\",\"Drinking\":\"No\"}", "Race": "Black", "Gender": "Female", "Age": 80}
---------------------------------------------------------------------------------------------------
******GROUND_TRUTH******
'json{"age": "80", "gender": "F", "living_situation": "lives alone though her family is in the area", "relationship_status": ""}'


In [ ]:
final_df.to_csv('output.csv', index=False)

# Inference for Clinical Entity

**Load uploded Model from hugging face**

In [24]:
peft_model_dir_1 = "NitinGCJ/ai-quest-p2"  # Replace with actual model name or path

# Load base LLM model and tokenizer
trained_model_1 = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir_1,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    # use_auth_token=True  # Add if you need authentication
)
tokenizer_1 = AutoTokenizer.from_pretrained(peft_model_dir)

adapter_config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

In [27]:
# No need to move the model to the device if it's using bitsandbytes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vect=[]
for i in range (2,len(dataset['test'])):
    
    dialogue = dataset['test'][i]['input']
    print(dialogue)
                               
    # Prepare the prompt
    prompt =f"""Act as an expert Analyst with over 20 years of experience in the Pharma and Healthcare industry. 
    For the following input, your task is to identify and extract the following entities: 
    'PROBLEM', 'ANATOMICAL_STRUCTURE', 'PROCEDURE', 'SEVERITY', 'LAB_RESULT', 'PROCEDURE_RESULT', 'LABORATORY_DATA', 
    'MEDICINE', 'MED_STRENGTH', 'MED_UNIT', 'MED_ROUTE', 'MED_FREQUENCY', 'SUBSTANCE_ABUSE', 'BODY_FUNCTION', 'BF_RESULT', and 'BODY_MEASUREMENT',etc. 
    Also, provide the start and end positions of these entities in the text.

    Use the following format for your response:
    {{\"entity_value\": \"Allergies\",\"entity_type\": \"PROBLEM\",\"start_position\": 164,\"end_position\": 173}}

    ### Extract Entities from the following text:
    {dialogue.strip()}

    ### Response:
    """
    
    input_ids = tokenizer_1(prompt, return_tensors='pt',truncation=True).input_ids.cuda()
    outputs = trained_model_1.generate(input_ids=input_ids, max_new_tokens=2000, )
    output= tokenizer_1.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
    print(output)                         

 {"entity_value": "Allergies", "entity_type": "PROBLEM", "start_position": 164, "end_position": 173}


1. "PROBLEM", "PROCEDURE", "SEVERITY", "LAB_RESULT", "LABORATORY_DATA", 
       "BODY_MEASUREMENT", "MEDICINE", "MED_STRENGTH", "MED_UNIT", "MED_ROUTE", "MED_FREQUENCY", 
       "SUBSTANCE_ABUSE", "BODY_FUNCTION", "BF_RESULT", "BODY_MEASUREMENT_UNIT", "BODY_MEASUREMENT_RESULT", "BODY_MEASUREMENT_STATUS", 
       "PROCEDURE_STATUS", "PROCEDURE_RESULT", "PROCEDURE_MEDICINE", "PROCEDURE_MED_STRENGTH", "PROCEDURE_MED_UNIT", "PROCEDURE_MED_ROUTE", "PROCEDURE_MED_FREQUENCY", 
       "BODY_MEASUREMENT_STATUS", "PROBLEM", "PROBLEM_RESULT", "PROBLEM_MEDICINE", "PROBLEM_MED_STRENGTH", "PROBLEM_MED_UNIT", "PROBLEM_MED_ROUTE", "PROBLEM_MED_FREQUENCY", 
       "PROCEDURE_MEDICINE", "PROCEDURE_MED_STRENGTH", "PROCEDURE_MED_UNIT", "PROCEDURE_MED_ROUTE", "PROCEDURE_MED_FREQUENCY", 
       "BODY_MEASUREMENT_STATUS", "PROCEDURE", "PROCEDURE_RESULT", "PROCEDURE_MEDICINE", "PROCEDURE_MED_STRENGTH", "PROC

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.



    {"entity_value": "PROBLEM", "entity_type": "PROBLEM", "start_position": 205, "end_position": 212,},
    {"entity_value": "ACQUIRED IMMUNODEFICIENCY SYNDROME", "entity_type": "PROBLEM", "start_position": 215, "end_position": 240,},
    {"entity_value": "PNEUMOCYSTIS CARINII PNEUMONIA", "entity_type": "PROBLEM", "start_position": 243, "end_position": 263,},
    {"entity_value": "ADULT RESPIRATORY DISTRESS SYNDROME", "entity_type": "PROBLEM", "start_position": 265, "end_position": 284,},
    {"entity_value": "DISEASE", "entity_type": "PROBLEM", "start_position": 286, "end_position": 292,},
    {"entity_value": "HIV", "entity_type": "PROBLEM", "start_position": 293, "end_position": 296,},
    {"entity_value": "PCP", "entity_type": "PROBLEM", "start_position": 297, "end_position": 299,},
    {"entity_value": "Bactrim", "entity_type": "MEDICINE", "start_position": 300, "end_position": 306,},
    {"entity_value": "Pulmonary", "entity_type": "PROBLEM", "start_position": 317, "end_position